# Phase 1: Data Preprocessing & Feature Engineering

**Objective:** Prepare the Amazon products dataset for machine learning modeling

---

## Steps:
1. Load and clean data
2. Handle missing values
3. Feature engineering
4. Encode categorical variables
5. Scale numerical features
6. Save preprocessed data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
import pickle

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Data

In [2]:
# Load dataset
df = pd.read_csv('amazon_products_sales_data_cleaned.csv')

print(f"Original dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nMissing values:\n{df.isnull().sum()}")

Original dataset shape: (42675, 17)

Columns: ['product_title', 'product_rating', 'total_reviews', 'purchased_last_month', 'discounted_price', 'original_price', 'is_best_seller', 'is_sponsored', 'has_coupon', 'buy_box_availability', 'delivery_date', 'sustainability_tags', 'product_image_url', 'product_page_url', 'data_collected_at', 'product_category', 'discount_percentage']

Missing values:
product_title               0
product_rating           1024
total_reviews            1024
purchased_last_month    10511
discounted_price         2062
original_price           2062
is_best_seller              0
is_sponsored                0
has_coupon                  0
buy_box_availability    14653
delivery_date           11983
sustainability_tags     39267
product_image_url           0
product_page_url         2069
data_collected_at           0
product_category            0
discount_percentage      2062
dtype: int64


## 2. Handle Target Variable Missing Values

Since our target is `purchased_last_month`, we need to remove rows where it's missing.

In [3]:
# Remove rows where target variable is missing
print(f"Rows before removing target nulls: {len(df)}")
df = df.dropna(subset=['purchased_last_month'])
print(f"Rows after removing target nulls: {len(df)}")
print(f"Rows removed: {42675 - len(df)}")

Rows before removing target nulls: 42675
Rows after removing target nulls: 32164
Rows removed: 10511


## 3. Select and Clean Features

Remove unnecessary columns (URLs, timestamps) and focus on predictive features.

In [4]:
# Drop unnecessary columns
columns_to_drop = [
    'product_title',  # Text - would need NLP processing
    'product_image_url',  # Not predictive
    'product_page_url',  # Not predictive
    'data_collected_at',  # Timestamp - all same date
    'delivery_date'  # Too many missing values and not reliable
]

df = df.drop(columns=columns_to_drop)
print(f"Remaining columns: {df.columns.tolist()}")
print(f"\nDataset shape: {df.shape}")

Remaining columns: ['product_rating', 'total_reviews', 'purchased_last_month', 'discounted_price', 'original_price', 'is_best_seller', 'is_sponsored', 'has_coupon', 'buy_box_availability', 'sustainability_tags', 'product_category', 'discount_percentage']

Dataset shape: (32164, 12)


## 4. Handle Missing Values in Features

In [5]:
# Check missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing_Count': missing, 'Missing_Percentage': missing_pct})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
print("Missing values:")
print(missing_df)

Missing values:
                      Missing_Count  Missing_Percentage
sustainability_tags           29422               91.47
buy_box_availability           8662               26.93
discounted_price               1860                5.78
original_price                 1860                5.78
discount_percentage            1860                5.78
product_rating                   76                0.24
total_reviews                    76                0.24


In [6]:
# Handle sustainability_tags (too many missing - drop or create flag)
df['has_sustainability'] = df['sustainability_tags'].notna().astype(int)
df = df.drop('sustainability_tags', axis=1)

# Handle buy_box_availability (many missing - create flag)
df['has_buy_box_info'] = df['buy_box_availability'].notna().astype(int)
df['buy_box_availability'] = df['buy_box_availability'].fillna('Unknown')

# Handle numerical features with missing values
# For rating and reviews - fill with median (neutral approach)
df['product_rating'] = df['product_rating'].fillna(df['product_rating'].median())
df['total_reviews'] = df['total_reviews'].fillna(df['total_reviews'].median())

# For price features - fill with median
df['discounted_price'] = df['discounted_price'].fillna(df['discounted_price'].median())
df['original_price'] = df['original_price'].fillna(df['original_price'].median())
df['discount_percentage'] = df['discount_percentage'].fillna(0)  # No discount if missing

print("\nMissing values after handling:")
print(df.isnull().sum().sum())


Missing values after handling:
0


## 5. Feature Engineering

In [7]:
# Create new features

# Price-based features
df['discount_amount'] = df['original_price'] - df['discounted_price']
df['price_ratio'] = df['discounted_price'] / (df['original_price'] + 1)  # +1 to avoid division by zero

# Rating and review features
df['rating_review_interaction'] = df['product_rating'] * np.log1p(df['total_reviews'])
df['log_total_reviews'] = np.log1p(df['total_reviews'])

# Log transform the target variable to reduce skewness
df['log_purchased_last_month'] = np.log1p(df['purchased_last_month'])

print("Engineered features created!")
print(f"\nNew feature columns: {[col for col in df.columns if col not in ['product_rating', 'total_reviews', 'purchased_last_month', 'discounted_price', 'original_price', 'discount_percentage']]}")

Engineered features created!

New feature columns: ['is_best_seller', 'is_sponsored', 'has_coupon', 'buy_box_availability', 'product_category', 'has_sustainability', 'has_buy_box_info', 'discount_amount', 'price_ratio', 'rating_review_interaction', 'log_total_reviews', 'log_purchased_last_month']


## 6. Encode Categorical Variables

In [8]:
# Binary encoding for badge/promotional features
df['is_best_seller'] = (df['is_best_seller'] == 'Best Seller').astype(int)
df['is_sponsored'] = (df['is_sponsored'] == 'Sponsored').astype(int)
df['has_coupon'] = (df['has_coupon'] != 'No Coupon').astype(int)

# Encode buy_box_availability
df['has_add_to_cart'] = (df['buy_box_availability'] == 'Add to cart').astype(int)
df = df.drop('buy_box_availability', axis=1)

# One-hot encode product_category (if not too many categories)
print(f"\nNumber of unique categories: {df['product_category'].nunique()}")
print(f"Category value counts:\n{df['product_category'].value_counts()}")

# One-hot encode categories
category_dummies = pd.get_dummies(df['product_category'], prefix='category', drop_first=True)
df = pd.concat([df, category_dummies], axis=1)
df = df.drop('product_category', axis=1)

print(f"\nFinal dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")


Number of unique categories: 15
Category value counts:
product_category
Laptops                6634
Phones                 5806
Other Electronics      5425
Power & Batteries      2783
Cameras                2599
Chargers & Cables      1546
TV & Display           1373
Headphones              993
Speakers                959
Storage                 958
Printers & Scanners     876
Networking              846
Gaming                  666
Wearables               447
Smart Home              253
Name: count, dtype: int64

Final dataset shape: (32164, 31)

Columns: ['product_rating', 'total_reviews', 'purchased_last_month', 'discounted_price', 'original_price', 'is_best_seller', 'is_sponsored', 'has_coupon', 'discount_percentage', 'has_sustainability', 'has_buy_box_info', 'discount_amount', 'price_ratio', 'rating_review_interaction', 'log_total_reviews', 'log_purchased_last_month', 'has_add_to_cart', 'category_Chargers & Cables', 'category_Gaming', 'category_Headphones', 'category_Laptops', 'ca

## 7. Prepare Train/Test Split

In [9]:
# Separate features and targets
# We'll keep both original and log-transformed target for different models

# Original target
y_original = df['purchased_last_month'].copy()

# Log-transformed target (better for models sensitive to outliers)
y_log = df['log_purchased_last_month'].copy()

# Features (drop both target variables)
X = df.drop(['purchased_last_month', 'log_purchased_last_month'], axis=1)

print(f"Features shape: {X.shape}")
print(f"Target (original) shape: {y_original.shape}")
print(f"Target (log) shape: {y_log.shape}")
print(f"\nFeature columns: {X.columns.tolist()}")

Features shape: (32164, 29)
Target (original) shape: (32164,)
Target (log) shape: (32164,)

Feature columns: ['product_rating', 'total_reviews', 'discounted_price', 'original_price', 'is_best_seller', 'is_sponsored', 'has_coupon', 'discount_percentage', 'has_sustainability', 'has_buy_box_info', 'discount_amount', 'price_ratio', 'rating_review_interaction', 'log_total_reviews', 'has_add_to_cart', 'category_Chargers & Cables', 'category_Gaming', 'category_Headphones', 'category_Laptops', 'category_Networking', 'category_Other Electronics', 'category_Phones', 'category_Power & Batteries', 'category_Printers & Scanners', 'category_Smart Home', 'category_Speakers', 'category_Storage', 'category_TV & Display', 'category_Wearables']


## 8. Train-Test Split

In [10]:
# Split data (80/20)
X_train, X_test, y_train_original, y_test_original = train_test_split(
    X, y_original, test_size=0.2, random_state=42
)

# Also split for log-transformed target
_, _, y_train_log, y_test_log = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"\nTraining set percentage: {X_train.shape[0] / len(X) * 100:.1f}%")
print(f"Test set percentage: {X_test.shape[0] / len(X) * 100:.1f}%")

Training set size: 25731
Test set size: 6433

Training set percentage: 80.0%
Test set percentage: 20.0%


## 9. Scale Numerical Features

In [11]:
# Fit scaler on training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame to keep column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Features scaled successfully!")
print(f"\nScaled training data shape: {X_train_scaled.shape}")
print(f"Scaled test data shape: {X_test_scaled.shape}")

Features scaled successfully!

Scaled training data shape: (25731, 29)
Scaled test data shape: (6433, 29)


## 10. Save Preprocessed Data and Scaler

In [12]:
# Save preprocessed data
X_train_scaled.to_csv('X_train_scaled.csv', index=False)
X_test_scaled.to_csv('X_test_scaled.csv', index=False)

# Save original (unscaled) data as well for tree-based models
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)

# Save targets
y_train_original.to_csv('y_train_original.csv', index=False, header=['purchased_last_month'])
y_test_original.to_csv('y_test_original.csv', index=False, header=['purchased_last_month'])
y_train_log.to_csv('y_train_log.csv', index=False, header=['log_purchased_last_month'])
y_test_log.to_csv('y_test_log.csv', index=False, header=['log_purchased_last_month'])

# Save scaler for future use
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save feature names
feature_names = X_train.columns.tolist()
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

print("✓ All preprocessed data saved successfully!")
print("\nFiles created:")
print("  - X_train_scaled.csv (scaled features for linear models)")
print("  - X_test_scaled.csv")
print("  - X_train.csv (unscaled features for tree models)")
print("  - X_test.csv")
print("  - y_train_original.csv")
print("  - y_test_original.csv")
print("  - y_train_log.csv")
print("  - y_test_log.csv")
print("  - scaler.pkl")
print("  - feature_names.pkl")

✓ All preprocessed data saved successfully!

Files created:
  - X_train_scaled.csv (scaled features for linear models)
  - X_test_scaled.csv
  - X_train.csv (unscaled features for tree models)
  - X_test.csv
  - y_train_original.csv
  - y_test_original.csv
  - y_train_log.csv
  - y_test_log.csv
  - scaler.pkl
  - feature_names.pkl


## 11. Summary Statistics

In [13]:
# Display summary
print("=" * 60)
print("PREPROCESSING SUMMARY")
print("=" * 60)
print(f"\nOriginal dataset: 42,675 rows × 17 columns")
print(f"After removing missing targets: {len(X)} rows")
print(f"Final feature count: {X_train.shape[1]} features")
print(f"\nTraining samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"\nTarget variable (original):")
print(f"  Train - Mean: {y_train_original.mean():.2f}, Median: {y_train_original.median():.2f}")
print(f"  Test  - Mean: {y_test_original.mean():.2f}, Median: {y_test_original.median():.2f}")
print(f"\nTarget variable (log-transformed):")
print(f"  Train - Mean: {y_train_log.mean():.2f}, Median: {y_train_log.median():.2f}")
print(f"  Test  - Mean: {y_test_log.mean():.2f}, Median: {y_test_log.median():.2f}")
print(f"\n✓ Data is ready for model training!")
print("=" * 60)

PREPROCESSING SUMMARY

Original dataset: 42,675 rows × 17 columns
After removing missing targets: 32164 rows
Final feature count: 29 features

Training samples: 25731
Test samples: 6433

Target variable (original):
  Train - Mean: 1312.64, Median: 200.00
  Test  - Mean: 1217.77, Median: 200.00

Target variable (log-transformed):
  Train - Mean: 5.40, Median: 5.30
  Test  - Mean: 5.39, Median: 5.30

✓ Data is ready for model training!


---

## Next Steps

Proceed to **Phase 2: Model Training** to train multiple regression models using this preprocessed data.